In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

BASE_PATH = "/Users/ananthu/AMRITA/Datathon_2025/Education_Dataset/"

FILES_2324 = {
    "facility": BASE_PATH + "100_fac.csv",
    "teacher": BASE_PATH + "100_tch.csv",
    "profile1": BASE_PATH + "100_prof1.csv",
    "profile2": BASE_PATH + "100_prof2.csv",
    "enrol1": BASE_PATH + "100_enr1.csv",
    "enrol2": BASE_PATH + "100_enr2.csv"
}

FILES_2425 = {
    "facility": BASE_PATH + "100_fac 2.csv",
    "teacher": BASE_PATH + "100_tch 2.csv",
    "profile1": BASE_PATH + "100_prof1 2.csv",
    "profile2": BASE_PATH + "100_prof2 2.csv",
    "enrol1": BASE_PATH + "100_enr1 2.csv",
    "enrol2": BASE_PATH + "100_enr2 2.csv"
}

In [4]:
def clean_facility(df):
    binary_cols = ['electricity_availability', 'internet', 'ict_lab_yn',
                   'library_availability', 'playground_available', 
                   'handwash_facility_for_meal']
    for col in binary_cols:
        if col in df.columns:
            df[col] = df[col].replace({1:1, 2:0, 3:0})
    return df

def clean_teacher(df):
    cols_needed = ['pseudocode','total_tch','male','female']
    return df[cols_needed].dropna(subset=['pseudocode'])

def clean_profile(df):
    cols_needed = ['pseudocode','state','district']
    return df[cols_needed]

def clean_enrolment(df):
    df['total_students'] = df.filter(like='_b').sum(axis=1) + df.filter(like='_g').sum(axis=1)
    df['boys_enrolled'] = df.filter(like='_b').sum(axis=1)
    df['girls_enrolled'] = df.filter(like='_g').sum(axis=1)
    return df[['pseudocode','total_students','boys_enrolled','girls_enrolled']]

In [5]:
def process_year(file_dict, year_label):
    facility = clean_facility(pd.read_csv(file_dict["facility"], low_memory=False))
    teacher = clean_teacher(pd.read_csv(file_dict["teacher"], low_memory=False))
    profile1 = clean_profile(pd.read_csv(file_dict["profile1"], low_memory=False))
    enrolment = clean_enrolment(pd.read_csv(file_dict["enrol1"], low_memory=False))

    merged = facility.merge(teacher, on='pseudocode', how='left') \
                     .merge(enrolment, on='pseudocode', how='left') \
                     .merge(profile1, on='pseudocode', how='left')

    district_df = merged.groupby(['state','district'], as_index=False).mean(numeric_only=True)
    district_df['year'] = year_label
    return district_df

In [6]:
def compute_indices(df):
    scaler = MinMaxScaler()

    if 'laptop' in df.columns:
        df['digital_device_avg'] = scaler.fit_transform(df[['laptop']].fillna(0))
    elif 'desktop' in df.columns:
        df['digital_device_avg'] = scaler.fit_transform(df[['desktop']].fillna(0))
    else:
        df['digital_device_avg'] = 0

    def col(df, name_list):
        for n in name_list:
            if n in df.columns:
                return df[n]
        return pd.Series(0, index=df.index)

    df['basic_infra_index'] = (
        col(df, ['electricity_availability']) +
        col(df, ['total_girls_func_toilet', 'total_boys_func_toilet'])/2 +
        col(df, ['tap_fun_yn', 'pack_water_fun_yn', 'hand_pump_fun_yn'])/3 +
        col(df, ['furniture_availability', 'furniture_availability_rate']) +
        col(df, ['playground_available', 'playground_alt_yn'])
    ) / 5

    df['digital_infra_index'] = (
        col(df, ['internet']) +
        col(df, ['ict_lab_yn', 'comp_ict_lab_yn']) +
        col(df, ['comp_lab_cond', 'computer_lab_condition']) +
        col(df, ['smart_class_rate', 'smart_class_tv_tot']) +
        df['digital_device_avg']
    ) / 5

    df['learning_env_index'] = (
        col(df, ['library_availability']) +
        col(df, ['handwash_facility_for_meal']) +
        col(df, ['medical_checkups']) +
        col(df, ['availability_ramps', 'ramps_availability_rate']) +
        col(df, ['phy_lab_cond', 'chem_lab_cond', 'bio_lab_cond']).mean()
    ) / 5

    df['infrastructure_index'] = df[['basic_infra_index',
                                     'digital_infra_index',
                                     'learning_env_index']].mean(axis=1)

    if 'total_tch' in df.columns:
        df['PTR'] = df['total_students'] / df['total_tch']
    else:
        df['PTR'] = np.nan

    if 'girls_enrolled' in df.columns and 'boys_enrolled' in df.columns:
        df['GPI'] = df['girls_enrolled'] / df['boys_enrolled']
    else:
        df['GPI'] = np.nan

    return df

In [7]:
df_2324 = process_year(FILES_2324, '2023-24')
df_2425 = process_year(FILES_2425, '2024-25')

df_2324 = compute_indices(df_2324)
df_2425 = compute_indices(df_2425)

combined = pd.concat([df_2324, df_2425], ignore_index=True)
combined = combined.sort_values(['state', 'district', 'year'])

combined = combined.drop_duplicates(subset=['state','district','year'], keep='first')

retention = (
    combined
    .groupby(['state','district'], as_index=False, group_keys=False)
    .apply(lambda g: g.assign(Retention_Rate=(g['total_students'].pct_change()+1)*100))
)

combined = retention.copy()

combined['Dropout_Rate'] = 100 - combined['Retention_Rate']

combined['Retention_Rate'] = combined['Retention_Rate'].fillna(100)
combined['Dropout_Rate'] = combined['Dropout_Rate'].fillna(0)

print("Retention and Dropout computed successfully.")
print("Rows:", combined.shape[0])

Retention and Dropout computed successfully.
Rows: 1570


/var/folders/fp/w6yn5n6d4cb299k1w5hy5xgc0000gn/T/ipykernel_18100/2525095314.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.assign(Retention_Rate=(g['total_students'].pct_change()+1)*100))


In [8]:
combined['GER_primary'] = (combined['total_students'] / combined['total_students'].max()) * 100

if {'girls_enrolled', 'boys_enrolled'}.issubset(combined.columns):
    combined['GPI'] = combined['girls_enrolled'] / combined['boys_enrolled']
else:
    combined['GPI'] = np.nan

def gpi_category(x):
    if pd.isna(x): return "Unknown"
    if 0.98 <= x <= 1.02: return "Equal"
    elif x < 0.98: return "Low Female Participation"
    else: return "High Female Participation"

combined['GPI_Category'] = combined['GPI'].apply(gpi_category)

if {'total_students', 'total_tch'}.issubset(combined.columns):
    combined['PTR'] = combined['total_students'] / combined['total_tch']
else:
    combined['PTR'] = np.nan

def ptr_category(x):
    if pd.isna(x): return "Unknown"
    if x <= 30: return "Optimal"
    elif x <= 40: return "Moderate"
    else: return "Overcrowded"

combined['PTR_Category'] = combined['PTR'].apply(ptr_category)

combined['GER_primary'] = combined['GER_primary'].fillna(0)
combined['GPI'] = combined['GPI'].fillna(1)
combined['PTR'] = combined['PTR'].fillna(35)

print("GER, GPI, PTR and categories computed successfully.")

GER, GPI, PTR and categories computed successfully.


In [9]:
print("\nAverage GER:", round(combined['GER_primary'].mean(), 2))
print("Average GPI:", round(combined['GPI'].mean(), 2))
print("Average PTR:", round(combined['PTR'].mean(), 2))
print("\nTop 5 states by Infra Index:")
print(combined.groupby('state')['infrastructure_index'].mean().sort_values(ascending=False).head(5))


Average GER: 29.23
Average GPI: 1.05
Average PTR: 5.76

Top 5 states by Infra Index:
state
CHANDIGARH     2.219382
DELHI          1.818905
LAKSHADWEEP    1.568100
KERALA         1.566059
PUDUCHERRY     1.543731
Name: infrastructure_index, dtype: float64


In [10]:
export_cols = [
    'state','district','year',
    'basic_infra_index','digital_infra_index','learning_env_index','infrastructure_index',
    'GER_primary','GPI','GPI_Category','PTR','PTR_Category',
    'Retention_Rate','Dropout_Rate'
]

export_cols = [c for c in export_cols if c in combined.columns]
final_df = combined[export_cols].dropna(subset=['state','district']).reset_index(drop=True)

final_df['infrastructure_index'] = final_df['infrastructure_index'].clip(0,1)
final_df['GER_primary'] = final_df['GER_primary'].clip(0,150)
final_df['GPI'] = final_df['GPI'].clip(0,2)
final_df['PTR'] = final_df['PTR'].clip(0,80)
final_df['Retention_Rate'] = final_df['Retention_Rate'].clip(0,150)
final_df['Dropout_Rate'] = final_df['Dropout_Rate'].clip(-50,100)

OUT_PATH = BASE_PATH + "District_Level_Education_Infra_Final.csv"
final_df.to_csv(OUT_PATH, index=False)
print(f"Exported Tableau-ready file: {OUT_PATH}")
print(f"Rows: {final_df.shape[0]}, Columns: {final_df.shape[1]}")

print("\n=== DATA VALIDATION SUMMARY ===")
print("Average Infrastructure Index:", round(final_df['infrastructure_index'].mean(), 3))
print("Average GER:", round(final_df['GER_primary'].mean(), 2))
print("Average PTR:", round(final_df['PTR'].mean(), 2))
print("Average GPI:", round(final_df['GPI'].mean(), 2))
print("\nRetention & Dropout Check (first few rows):")
print(final_df[['state','district','year','Retention_Rate','Dropout_Rate']].head(10))

Exported Tableau-ready file: /Users/ananthu/AMRITA/Datathon_2025/Education_Dataset/District_Level_Education_Infra_Final.csv
Rows: 1570, Columns: 14

=== DATA VALIDATION SUMMARY ===
Average Infrastructure Index: 0.99
Average GER: 29.23
Average PTR: 5.76
Average GPI: 1.05

Retention & Dropout Check (first few rows):
                       state                   district     year  \
0  ANDAMAN & NICOBAR ISLANDS                   ANDAMANS  2023-24   
1  ANDAMAN & NICOBAR ISLANDS                   ANDAMANS  2024-25   
2  ANDAMAN & NICOBAR ISLANDS  MIDDLE AND NORTH ANDAMANS  2023-24   
3  ANDAMAN & NICOBAR ISLANDS  MIDDLE AND NORTH ANDAMANS  2024-25   
4  ANDAMAN & NICOBAR ISLANDS                   NICOBARS  2023-24   
5  ANDAMAN & NICOBAR ISLANDS                   NICOBARS  2024-25   
6             ANDHRA PRADESH                 ANAKAPALLI  2023-24   
7             ANDHRA PRADESH                 ANAKAPALLI  2024-25   
8             ANDHRA PRADESH                  ANANTAPUR  2023-24   
9   

In [11]:
import pandas as pd

file_path = "/Users/ananthu/AMRITA/Datathon_2025/Education_Dataset/District_Level_Education_Infra_Final.csv"
df = pd.read_csv(file_path)

df = df.sort_values(['state', 'year'])

def add_yoy_growth(df, value_col, new_col):
    df[new_col] = (
        df.groupby('state')[value_col].pct_change() * 100
    ).round(2)
    return df

df = (
    df.pipe(add_yoy_growth, 'infrastructure_index', 'infra_growth_pct')
      .pipe(add_yoy_growth, 'digital_infra_index', 'digital_infra_growth_pct')
      .pipe(add_yoy_growth, 'GER_primary', 'GER_growth_pct')
      .pipe(add_yoy_growth, 'GPI', 'GPI_growth_pct')
      .pipe(add_yoy_growth, 'PTR', 'PTR_change_pct')
      .pipe(add_yoy_growth, 'Retention_Rate', 'Retention_change_pct')
)

df = df.fillna(0)

df.to_csv(file_path, index=False)

print("Existing file successfully updated with growth % columns!")
print("Added columns:", 
      ['infra_growth_pct', 'digital_infra_growth_pct', 
       'GER_growth_pct', 'GPI_growth_pct', 
       'PTR_change_pct', 'Retention_change_pct'])

Existing file successfully updated with growth % columns!
Added columns: ['infra_growth_pct', 'digital_infra_growth_pct', 'GER_growth_pct', 'GPI_growth_pct', 'PTR_change_pct', 'Retention_change_pct']
